# RAG
1. 문서 내용 읽기
    - https://python.langchain.com/v0.2/docs/integrations/document_loaders/
1. 문서 분할(쪼개기)
    - 문서 분할하지 않으면?
        - 토큰 수 초과로 답변을 생성하지 않을 수 있음
        - 문서 길이(input)가 길어서 답변 생성에 시간 소요됨 - > 답변 늦어짐
1. 임베딩 - 벡터 데이터베이스에 저장
1. 질문이 있으면 벡터 데이터베이스에 유사도 검색(코사인 유사도)
1. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [ ]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import Docx2txtLoader
from dotenv import load_dotenv


## 1, 2. 문서 Microsoft Word 파일 읽고 분할 한번에 
loader = Docx2txtLoader("law_1.docx")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

data_list = loader.load_and_split(text_splitter=text_splitter)


# 3. 임베딩 -> 벡터 데이터베이스에 저장 
## 환경변수 읽어오기
load_dotenv()

## 3-1. 임베딩 
embedding = OpenAIEmbeddings(
    model="text-embedding-3-large",
)

## 3-2 벡터 데이터베이스에 저장 
## [방법1] inmemory
database = Chroma.from_documents(
    documents=data_list,
    embedding=embedding,
)


## 4. 사용자 질문 + 벡터 데이터베이스에서 유사도 검색
query = "전세사기피해자 금융지원에 대해 설명해주세요."

## 벡터 데이터베이스에서 유사도 검색(기본이 4개씩)
retrieved_docs = database.similarity_search(query=query)


## 5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 
## 5-1 프롬프트 작성
prompt = '''
[identity]
-당신은 전세사기피해 법률 전문가입니다
-[context]를 참고해 사용자의 질문에 답변해주세요. 

[context]
{retrieved_docs} 

Question : {query}

'''

formatted_prompt = prompt.format(
    retrieved_docs=retrieved_docs,
    query=query,
)

## 5-2 LLM 모델 생성, ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    model_name='gpt-4.1-nano'
)

aimessage = llm.invoke(formatted_prompt)
aimessage

[Document(metadata={'source': 'law_1.docx'}, page_content='전세사기피해자 지원 및 주거안정에 관한 특별법 ( 약칭: 전세사기피해자법 )\n\n[시행 2024. 11. 11.] [법률 제20429호, 2024. 9. 10., 일부개정]\n\n\n\n국토교통부(피해지원총괄과) 044-201-5233, 5234\n\n\n\n       제1장 총칙 \n\n제1조(목적) 이 법은 전세사기로 피해를 입은 임차인에게 경ㆍ공매 절차 및 조세 징수 등에 관한 특례를 부여함으로써 전세사기피해자를 지원하고 주거안정을 도모함을 목적으로 한다.\n\n제2조(정의) 이 법에서 사용하는 용어의 정의는 다음과 같다. <개정 2024. 9. 10.>\n\n  1. “주택”이란 「주택임대차보호법」 제2조에 따른 주거용 건물(공부상 주거용 건물이 아니라도 임대차계약 체결 당시 임대차목적물의 구조와 실질이 주거용 건물이고 임차인의 실제 용도가 주거용인 경우를 포함한다)을 말한다.\n\n  2. “임대인등”이란 임대인 또는 다음 각 목의 어느 하나에 해당하는 자를 말한다.\n\n    가. 임대인의 대리인, 그 밖에 임대인을 위하여 주택의 임대에 관하여 업무를 처리하는 자\n\n    나. 임대인의 의뢰를 받은 공인중개사(중개보조인을 포함한다)\n\n    다. 임대인을 위하여 임차인을 모집하는 자(그 피고용인을 포함한다)\n\n    라. 다수 임대인의 배후에 있는 동일인\n\n    마. 라목의 동일인이 지배하거나 경제적 이익을 공유하는 조직\n\n    바. 라목의 동일인이나 마목의 조직을 배후에 둔 다수의 임대인\n\n  3. “전세사기피해자”란 제3조의 요건을 모두 갖춘 자로서 제6조에 따른 전세사기피해지원위원회의 심의ㆍ의결을 거쳐 국토교통부장관이 결정한 임차인을 말한다.\n\n  4. “전세사기피해자등”이란 다음 각 목의 어느 하나에 해당하는 자를 말한다.\n\n    가. 전세사기피해자\n\n    나. 제3조제1항제1호ㆍ제3호 및 제4